In [2]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [3]:
#import earth data username and password from config.py file which is in the same folder with this jupyter notebook
import config
username=config.username
password=config.password

In [4]:
import time
import sys
import glob


In [5]:
from bs4 import BeautifulSoup
import os
import re
import requests
from osgeo import gdal,osr
import numpy


In [6]:
# main url of fire date data folder
url='https://e4ftl01.cr.usgs.gov/MOTA/MCD64A1.006/'

In [7]:
page=requests.get(url)

In [8]:
modis_page=BeautifulSoup(page.content,'html.parser')

In [ ]:
year_list=modis_page.find_all('a')[8:]

In [ ]:
#create yearly and monthly folder
for year in year_list[49:61]:
    year_info=int(str(year['href']).split('.')[0])
    year_path=main_folder+str(year_info)
    try:
        os.mkdir(year_path)       
    except:
        pass
    try:
        folder_path=str(year_path)+'/'+str(year['href'])
        print(folder_path)
        os.mkdir(folder_path)
    except:
        pass
    

In [12]:
import http.cookiejar as CookieJar
from urllib.parse import urlencode 
import urllib.request as urllib2

In [ ]:
# download folder of data which are save by monthly under this folder
main_folder='../data/Modis_Fire_Date/'
# starting index 1 because we want to start 2001.01 data

for year in year_list[52:61]:
    # In this script, year means month of the year. ex: <a href="2001.01.01/">2001.01.01/</a>
    
    # main url of fire date data folder
    url='https://e4ftl01.cr.usgs.gov/MOTA/MCD64A1.006/'
    url_year=url+str(year['href'])
    year_info=int(str(year['href']).split('.')[0])
    print(url_year)
    
    #if you want to download spesific year you can define year. if you want to give range between years>>
    # >  if year_info>2004 and year_info<2010  
    #if year_info<2018:
        #print(year_info)
        #continue
    
    page=requests.get(url_year)
    year_page=BeautifulSoup(page.content,'html.parser')
    links = year_page.findAll(href=re.compile("\.hdf$"))
    
    # check the dwonload path.
    # sample >>>  data/Modis_Fire_Date/2001/2001.01.01
    download_path=main_folder+str(year_info)+'/'+str(year['href'])
    
    for l in links:
        
        image_bool=False
        #with these block, we are run same url until it's finished
        while image_bool is False:
            try:
                url_img=url_year+l['href']
                image_path=download_path+l['href']
                if os.path.exists(image_path):
                    #if this file already exist in target file, skip it
                    image_bool=True
                    continue
                
                username = username
                password = password
                password_manager = urllib2.HTTPPasswordMgrWithDefaultRealm()
                password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)
                cookie_jar = CookieJar.CookieJar()
                opener = urllib2.build_opener(urllib2.HTTPBasicAuthHandler(password_manager),
                                              #urllib2.HTTPHandler(debuglevel=1),    # Uncomment these two lines to see
                                              #urllib2.HTTPSHandler(debuglevel=1),   # details of the requests/responses
                                              urllib2.HTTPCookieProcessor(cookie_jar))
                urllib2.install_opener(opener)
                request = urllib2.Request(url_img)
                response = urllib2.urlopen(request)
                 
                with open(image_path, 'wb') as f:
                    f.write(response.read())
                    image_bool=True
                    #print(image_path)
                    time.sleep(1)
            except:
                time.sleep(60)
                print ("Unexpected error:", sys.exc_info()[0])
                image_bool=False
                pass
    # script waits 2 minutes when a year is finished      
    time.sleep(120)   